In [1]:

from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
import numpy as np
import hdbscan
import pyarrow as pa
from rdkit.SimDivFilters import rdSimDivPickers


In [ ]:
#in_memory_stream = pa.input_stream('/home/lyg/data/pubchem/arrow/pubchem_sorted.arrow')
#opened_stream = pa.ipc.open_stream(in_memory_stream)
#table = opened_stream.read_all()

table = pa.ipc.RecordBatchFileReader(pa.memory_map('/home/lyg/data/pubchem/arrow/pubchem_best.arrow')).read_all()

# sort the table by the number of atoms as that will give easier examples first
# also sort by cid so better annotated cids are picked first
table = table.sort_by(['num_atoms','cid'])


In [3]:
radius=2
nBits=2048
max_records=6000000
thresh = 0.65 # <- minimum distance between cluster centroids. random threshold for morgan radius 2 is 0.65

mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=radius,fpSize=nBits)

def itertuples(table, chunk_size=1, max_records=None):
    for i in range(0, table.num_rows, chunk_size):
        if i >= max_records:
            break
        rows = table[i:i + chunk_size].to_pydict()
        yield rows


# Cluster with HDBSCAN using Jaccard (1 - Tanimoto)
def cluster_fingerprints(fps_numpy, min_cluster_size=2):
    clusterer = hdbscan.HDBSCAN(metric='jaccard', min_cluster_size=min_cluster_size, core_dist_n_jobs=16)
    labels = clusterer.fit_predict(fps_numpy)
    return labels, clusterer

# arr = np.zeros((len(table), nBits), dtype=bool)
arr = []
arr_index = []

for i, row in enumerate(itertuples(table, max_records=max_records)):
    smiles = row['smiles'][0]
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        # arr[i] = mfpgen.GetFingerprintAsNumPy(mol)
        arr.append(mfpgen.GetFingerprint(mol))
        arr_index.append(i)
    if i % 10000 == 0:
        print(i, smiles)

print(f"Finished computing fingerprints, clustering {len(arr)} molecules...")

lp = rdSimDivPickers.LeaderPicker()

picks = lp.LazyBitVectorPick(arr,len(arr),thresh)


# labels, clusterer = cluster_fingerprints(arr)


0 CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C


[14:23:28] WARNING: not removing hydrogen atom without neighbors
[14:23:28] WARNING: not removing hydrogen atom without neighbors


10000 CC(CCC(=O)NCCS(=O)(=O)O)[C@H]1CC[C@@H]2[C@@]1([C@H](C[C@H]3[C@H]2CC[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C
20000 CC1=C(C(=NO1)C2=C(C=CC=C2Cl)F)C(=O)N[C@H]3[C@@H]4N(C3=O)[C@H](C(S4)(C)C)C(=O)O


[14:23:34] WARNING: not removing hydrogen atom without neighbors
[14:23:35] WARNING: not removing hydrogen atom without neighbors
[14:23:35] WARNING: not removing hydrogen atom without neighbors
[14:23:35] WARNING: not removing hydrogen atom without neighbors
[14:23:35] WARNING: not removing hydrogen atom without neighbors
[14:23:35] WARNING: not removing hydrogen atom without neighbors
[14:23:36] WARNING: not removing hydrogen atom without neighbors
[14:23:36] WARNING: not removing hydrogen atom without neighbors


30000 CN(C)N=CC(=O)C1=CC=C(C=C1)OC2=CC=CC=C2
40000 C1=CC=C(C=C1)C2=NC3=C(C=CC=C3O2)CC(=O)O
50000 CCSC1=NC2=C(S1)C=C(C=C2)NCN3C(=NN=N3)C4=CC=C(C=C4)[N+](=O)[O-]


...

5850000 CC1=CC(=NN1CC2CCCN2C(=O)C3=CC=CC(=C3)C(=O)N)C
5860000 C1CC(CN(C1)C2=NC=C(C=C2)C(F)(F)F)C3OCCO3
5870000 COC1=CC(=C(C=C1)NS(=O)(=O)C2=CC(=C(C=C2)OC)C(=O)O)Cl
5880000 CC(C1=CC=CC=C1OC)NC(=O)C2=CC3=CC=CC=C3C(=O)N2
5890000 CC1=CC(=C(S1)C)CS(=O)(=O)C(C)C2=NC(=NO2)C(C)C
5900000 C1CNCCC1CCOC2=CC(=CC=C2)Br
5910000 CCOCC(=O)N[C@H]1CCCN(C1)CC2=CC(=C(C=C2)F)N(C=O)NC3=CN=C(C=C3)C


[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not removing hydrogen atom without neighbors
[16:25:26] WARNING: not r

5920000 C1=CC=C(C=C1)C2=C(N=C(N=C2Cl)C3=CC=CC=C3)NCCCCN4C=CN=C4[N+](=O)[O-]
5930000 C1CN(CCC1(CC2=CC=CC=C2C(F)(F)F)C(=O)O)CC3=CC=CO3
5940000 C1CN(CC(N1)C(=O)O)C(=O)C2=C(C=CC(=C2)N3C=CC=N3)Cl
5950000 C1CCN(C1)C(=O)CCN2CCCC(C2)CCC3=CC=C(C=C3)F
5960000 C1CN2C=C(N=C2CN1)C(=O)NCCC3=CSC(=N3)C4=NC=CN=C4
5970000 CN1C=CC2=C(C=CN=C21)C3=CC(=CC=C3)O
5980000 COCC1CCN(CC1)S(=O)(=O)N2CCCCC2CCC3=CC=CC=C3
5990000 CN1CCC(CC1)NC(=O)COC2=CC=C(C=C2)N(C)S(=O)(=O)C


[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not removing hydrogen atom without neighbors
[16:28:26] WARNING: not r

Finished computing fingerprints, clustering 5999996 molecules...


In [ ]:
# with open('cluster_labels.txt', 'w') as f:
#     for i in picks:
#         smiles = table['smiles'][i].as_py()
#         iupac = table['iupac'][i].as_py()
#         cid = table['cid'][i].as_py()
#         formula = table['formula'][i].as_py()
#         num_atoms = table['num_atoms'][i].as_py()
#         # f.write(f"{smiles}\t{iupac}\t{labels[i]}\n")
#         f.write(f"{cid}\t{smiles}\t{iupac}\t{formula}\t{num_atoms}\n")

def convert_string_view_to_string(table):
    for column in table.schema:
        if pa.types.is_string_view(column.type):
            # Recreate the column as pa.string
            table = table.set_column(
                table.schema.get_field_index(column.name),
                column.name,
                pa.array(table[column.name].to_pylist(), type=pa.string())
            )
    return table

table = table.take(pa.array(picks))

# huggingface only supports string, not string_view
table = convert_string_view_to_string(table)

with pa.OSFile('pubchem_best_cluster.arrow', 'wb') as sink:
    with pa.RecordBatchFileWriter(sink, table.schema) as writer:
        writer.write_table(table)
print(f"Finished writing {len(picks)} picks to pubchem_best_cluster.arrow")